In [16]:
import pandas as pd

# 打开JSON文件并将其解析为Python字典
# with open('verification.json', 'r') as file:
data = pd.read_json('verification.json')

In [17]:
data.head()

,title,link,abstract,subjects,authors,published,label
0,Identity Inference from CLIP Models using Only...,http://arxiv.org/abs/2405.14517v1,The widespread usage of large-scale multimodal...,"[cs.LG, cs.CR]","[Songze Li, Ruoxi Cheng, Xiaojun Jia]",2024-05-23T12:54:25Z,CLIP
1,Tuning-free Universally-Supervised Semantic Se...,http://arxiv.org/abs/2405.14294v1,This work presents a tuning-free semantic segm...,[cs.CV],"[Xiaobo Yang, Xiaojin Gong]",2024-05-23T08:13:52Z,CLIP
2,RET-CLIP: A Retinal Image Foundation Model Pre...,http://arxiv.org/abs/2405.14137v1,The Vision-Language Foundation model is increa...,[cs.CV],"[Jiawei Du, Jia Guo, Weihang Zhang, Shengzhu Y...",2024-05-23T03:20:51Z,CLIP
3,A Lost Opportunity for Vision-Language Models:...,http://arxiv.org/abs/2405.14977v1,"In the realm of deep learning, maintaining mod...",[cs.CV],"[Mario Döbler, Robert A. Marsden, Tobias Raich...",2024-05-23T18:27:07Z,CLIP
4,Designing A Sustainable Marine Debris Clean-up...,http://arxiv.org/abs/2405.14815v1,Marine debris poses a significant ecological t...,"[cs.CV, I.4; H.4; J.6]","[Raymond Wang, Nicholas R. Record, D. Whitney ...",2024-05-23T17:28:23Z,CLIP


In [1]:
import os, json, torch
import pandas as pd
import numpy as np
import heapq
from time import sleep
from transformers import StoppingCriteria, StoppingCriteriaList

from transformers import LlamaForCausalLM, LlamaTokenizer, PreTrainedTokenizerFast
from datasets import load_dataset,load_from_disk
from collections import defaultdict
import pickle, json

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

save_dir = '/root/autodl-fs/llama-3-8b-instruct'

tokenizer = PreTrainedTokenizerFast.from_pretrained(save_dir, low_cpu_mem_usage=True)
model = LlamaForCausalLM.from_pretrained(save_dir, low_cpu_mem_usage=True, torch_dtype=torch.bfloat16)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id
model.cuda()
print("Good, load successful!")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Good, load successful!


In [6]:
class PriorityQueue:
    def __init__(self, max_size):
        self.queue = []
        self.max_size = max_size

    def push(self, item):
        try:
            if len(self.queue) == self.max_size:
                if item[0] > self.queue[0][0]:
                    removed_item = heapq.heappop(self.queue)
                    # print(f'Removed item with lower priority: {removed_item}')
                else:
                    return
            # print(f'Pushing item: {item}')
            heapq.heappush(self.queue, item)
        except TypeError as e:
            print(f"TypeError encountered: {e}. Item: {item}")
            raise e

    def pop(self):
        return heapq.heappop(self.queue)

    def peek(self):
        return self.queue[0]

    def __len__(self):
        return len(self.queue)

In [10]:
for filed in ["Related to CLIP","Related to Attention","Related to GNN","Related to RLHF","Related to optimization question"]:
    demand = filed
    print("\n"+demand)

    df = pd.read_json('verification.json')
    pq = PriorityQueue(max_size=10)

    for index, row in df.iterrows():
        item = (row['title'], row['label'])
        abstract = row['abstract']
        prompt = "Please judge that if this paper relates with all my demand: \n ##Abstract##\n{\n"
        prompt = prompt + abstract + "\n}\n"
        prompt = prompt + "##Demands##\n"
        prompt = prompt + demand
        prompt = prompt + "Does this paper relate to all my demands. Please answer directly Yes or No: Yes"

        input_ids = tokenizer(prompt, padding=True, return_tensors="pt").input_ids.to(device)
        with torch.no_grad():
            outputs = model(input_ids)
            logits = outputs.logits
            probs = torch.log_softmax(logits, dim=-1)    
        answer_ids = tokenizer(["Yes", "No"], return_tensors="pt").input_ids.to(device)
        last = probs[:,-2:-1,:]
        # print(last.shape)
        answer = answer_ids[:,-1].unsqueeze(0).unsqueeze(0)
        # print(answer.shape)
        gathered_probs = torch.gather(last, 2, answer)  
        gathered_probs = gathered_probs.detach().cpu()
        # print("Gathered probabilities:", gathered_probs)
        g_probs = np.array(gathered_probs.view(-1).detach())
        exp_logits = np.exp(g_probs)
        sum_exp_logits = np.sum(exp_logits)
        probabilities = exp_logits / sum_exp_logits
        # print("Yes prob: ", probabilities[0])
        yes_prob = float(probabilities[0])
        # item["prob"] = yes_prob
        # print((yes_prob, item))
        pq.push((yes_prob, item))
        
    while pq.queue:
        prob, item = pq.pop()
        print(prob,item[1])
        



Related to CLIP
0.7549149990081787 ATTENTION
0.7549149990081787 GNN
0.7879312038421631 CLIP
0.798186719417572 CLIP
0.8175744414329529 CLIP
0.8175744414329529 CLIP
0.8354834914207458 CLIP
0.8438951373100281 CLIP
0.851952850818634 CLIP
0.8596637845039368 CLIP

Related to Attention
0.7772998213768005 ATTENTION
0.7772998809814453 RLHF
0.7879312038421631 ATTENTION
0.7981867790222168 ATTENTION
0.8080672025680542 ATTENTION
0.8080672025680542 ATTENTION
0.808067262172699 ATTENTION
0.8175745010375977 ATTENTION
0.8175745010375977 ATTENTION
0.8354835510253906 GNN

Related to GNN
0.7662936449050903 GNN
0.7981867790222168 GNN
0.7981868386268616 GNN
0.8080672025680542 GNN
0.8267117738723755 GNN
0.8267117738723755 GNN
0.8267118334770203 GNN
0.8519527912139893 GNN
0.8519527912139893 GNN
0.8519527912139893 GNN

Related to RLHF
0.7549149990081787 ATTENTION
0.7662936449050903 RLHF
0.7879312038421631 RLHF
0.7879312038421631 RLHF
0.7981867790222168 RLHF
0.8080672025680542 RLHF
0.8080672025680542 RLHF
0.808